<a href="https://colab.research.google.com/github/ddodds42/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/DAVID_DODDS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

- [ ] Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

- [ ] Then, use that trained RNN to generate Shakespearean-ish text.

- [ ] Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

- [ ] Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running.

- [ ] Then, once you've got a proof of concept - start pushing it more!

In [0]:
from __future__ import print_function

from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

import pandas as pd

In [0]:
import requests

In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
url = 'https://raw.githubusercontent.com/ddodds42/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/spearshake.txt'

r = requests.get(url)

In [81]:
spearshake = r.text
print(spearshake[:100])


Contents



               THE SONNETS

               ALL’S WELL THAT ENDS WELL

        


In [82]:
type(spearshake)

str

In [84]:
spearshake0 = spearshake.replace('\n',' ')
spearshake0 = spearshake0.replace('\r',' ')
spearshake0[:100]

'  Contents                       THE SONNETS                   ALL’S WELL THAT ENDS WELL            '

In [9]:
spearshake0[:1000]

' Contents                   THE SONNETS                 ALL’S WELL THAT ENDS WELL                 THE TRAGEDY OF ANTONY AND CLEOPATRA                 AS YOU LIKE IT                 THE COMEDY OF ERRORS                 THE TRAGEDY OF CORIOLANUS                 CYMBELINE                 THE TRAGEDY OF HAMLET, PRINCE OF DENMARK                 THE FIRST PART OF KING HENRY THE FOURTH                 THE SECOND PART OF KING HENRY THE FOURTH                 THE LIFE OF KING HENRY THE FIFTH                 THE FIRST PART OF HENRY THE SIXTH                 THE SECOND PART OF KING HENRY THE SIXTH                 THE THIRD PART OF KING HENRY THE SIXTH                 KING HENRY THE EIGHTH                 KING JOHN                 THE TRAGEDY OF JULIUS CAESAR                 THE TRAGEDY OF KING LEAR                 LOVE’S LABOUR’S LOST                 THE TRAGEDY OF MACBETH                 MEASURE FOR MEASURE                 THE MERCHANT OF VENICE                 THE MERRY WIVES OF WINDSOR      

In [0]:
chars = list(set(spearshake0))

In [0]:
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [12]:
len(chars)

100

In [0]:
maxlen = 40
step = 5

In [0]:
encoded = [char_int[c] for c in spearshake0]

In [0]:
sequences = []
next_char = []

for i in range(0, len(encoded) - maxlen, step):
  sequences.append(encoded[i : i+maxlen])
  next_char.append(encoded[i+maxlen])

In [16]:
len(sequences)

1110221

In [0]:
X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

In [18]:
X.shape, y.shape

((1110221, 40, 100), (1110221, 100))

In [0]:
for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    X[i,t,char] = 1
  
  y[i, next_char[i]] = 1

In [20]:
X.shape, y.shape

((1110221, 40, 100), (1110221, 100))

In [21]:
len(X[0,0])

100

In [22]:
X[0,0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [23]:
len(y[0])

100

In [24]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [0]:
def sample(preds):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / 1
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
  print()
  print('==== Generating text after Epoch %d' % epoch)
  start_index = random.randint(0, len(spearshake0) - maxlen - 1)
  generated = ''
  sentence = spearshake0[start_index: start_index+maxlen]
  generated += sentence
  print('==== Generating with seed: "'+ sentence +'"')
  sys.stdout.write(generated)
  for i in range(400):
    X_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      X_pred[0, t, char_int[char]] = 1
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = int_char[next_index]
    sentence = sentence[1:]+next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [30]:
model.fit(
    X, y, batch_size=256, epochs=10, #validation_split = 0.2,
    callbacks=[print_callback]
)

Epoch 1/10
4337/4337 [==============================] - ETA: 0s - loss: 1.9405
==== Generating text after Epoch 0
==== Generating with seed: "se thee.   COSTARD. O, Marry me to one F"
se thee.   COSTARD. O, Marry me to one Freme quemin.—Sit wath misple, in thee herse of thou ind nown, sipapiend you to cincry ant thus coutias, And,  bath munse Nope-and praelycl..      BE NEN. AN, Gresender CaRsing of the PrRancurie _atwares   [Leapimy Eater did._] Kind I mone oun ow? Mionos thee and, Whiling fail must, Whow whink with.   CLOLWILES And a Sneened Chiles and Evening at in and Aid forthis Encille in ties gonaunce ot at li
4337/4337 [==============================] - 47s 11ms/step - loss: 1.9405
Epoch 2/10
4333/4337 [============================>.] - ETA: 0s - loss: 1.8243
==== Generating text after Epoch 1
==== Generating with seed: "ke no strain But that Achilles, were his"
ke no strain But that Achilles, were his be, not, apin brow wall not heartongely of yer. ALBETO. No treengging thot b

In [0]:
pred = model.predict(X[[0]])

In [43]:
pred[0]

array([4.28326275e-05, 1.11069879e-03, 1.73306209e-03, 2.63820084e-06,
       1.47657311e-05, 4.34391259e-04, 3.64584941e-03, 4.02148766e-03,
       4.07605711e-03, 6.88358676e-04, 2.59594731e-06, 4.75905603e-04,
       1.25139747e-02, 2.57493026e-04, 2.81900610e-03, 1.98513753e-05,
       1.68040697e-03, 4.84629010e-04, 8.37468033e-05, 5.17744114e-09,
       2.35273107e-03, 7.95037746e-02, 2.24698684e-03, 9.94355767e-04,
       4.21971072e-06, 9.18396469e-03, 3.52732809e-06, 1.67343984e-07,
       3.22475610e-03, 5.46023576e-03, 2.80694053e-07, 2.76742945e-03,
       1.10706512e-03, 5.94337878e-04, 1.38332474e-03, 1.50723890e-05,
       1.40917522e-03, 4.04226239e-06, 4.07532061e-04, 8.91561271e-04,
       7.21118195e-05, 5.10165701e-05, 4.95659291e-09, 5.69546901e-05,
       4.06187406e-04, 3.50763514e-07, 6.13248408e-01, 1.40753221e-02,
       1.34512593e-04, 1.41478807e-01, 5.37898450e-04, 1.82140575e-05,
       3.41415126e-03, 1.36435870e-02, 1.25670352e-03, 3.68009169e-05,
      

In [44]:
pred[0].sum()

1.0

In [0]:
def joust(integer):
  start_index = random.randint(0, len(spearshake0) - maxlen - 1)
  generated = ''
  sentence = spearshake0[start_index: start_index+maxlen]
  seed_text = 'Seed text: "'+ sentence +'"'
  # sys.stdout.write(generated)
  for i in range(integer):
    X_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      X_pred[0, t, char_int[char]] = 1
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = int_char[next_index]
    generated += next_char
    sentence = sentence[1:]+next_char
  return print(seed_text, '\n', 'Shaken Speare:', generated)

In [60]:
joust(40)

Seed text: "s set our men in order,     And issue fo" 
 Shaken Speare: r tooss did love you. Here is before the


In [62]:
joust(400)

Seed text: "and trembled at the ill neighbourhood.  " 
 Shaken Speare:  CLEOPATRIUS. In merite.     A beging end that so hear     Sirtain since word not sir of the Pance,     and Grief, thy honour King.  MERNA. I’en him and the leavence sud Worvey an back     Thou his write me.   FORD. Now! Ludied, then you shall issoment the speerer, And raugh to sweete ourgetice subland,     And pales ware’d the fastes leave For to thy     A will sloens thought me the speed and  To


This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

- [x] Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

- [x] Then, use that trained RNN to generate Shakespearean-ish text.

- [x] Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

- [x] Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running.

- [x] Then, once you've got a proof of concept - start pushing it more!

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN